In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from shapely.geometry import Point
from fibre.optics import *

In [ ]:
inc_angle = np.radians(60.)
n_inc = 1.5
n_trans = 1.0j
trans_angle = snell(n_inc, inc_angle, n_trans)
print(trans_angle)

r_s = fresnel_r_s(n_inc, inc_angle, n_trans, trans_angle)
r_p = fresnel_r_p(n_inc, inc_angle, n_trans, trans_angle)

t_s = fresnel_t_s(n_inc, inc_angle, n_trans, trans_angle)
t_p = fresnel_t_p(n_inc, inc_angle, n_trans, trans_angle)

R = (np.abs(r_s)**2 + np.abs(r_p)**2)*0.5

#inc_angle = np.radians(inc_angle)
#trans_angle = np.radians(trans_angle)

T = (np.abs(t_s)**2 + np.abs(t_p)**2)*0.5* np.real(n_trans*np.cos(trans_angle)/(n_inc*np.cos(inc_angle)))

center_x = 0.
center_y = 0.

left_x = -np.cos(inc_angle)*2
left_y_lower = -np.sin(inc_angle)*2
left_y_upper = np.sin(inc_angle)*2

right_x = np.cos(trans_angle)*2
right_y = np.sin(trans_angle)*2

x_inc = [left_x, center_x]
y_inc = [left_y_lower, center_y]

x_ref = [center_x, left_x]
y_ref = [center_y, left_y_upper]

x_trn = [center_x, right_x]
y_trn = [center_y, right_y]

print(R,T,R+T)

In [ ]:
np.degrees(np.real(np.arcsin((n_inc/n_inc)*np.clip(np.sin(np.radians(60.)), -1., 1.))))

In [ ]:
def make_color(r,g,b):
    color_str = "{}, {}, {}".format(r,g,b)
    return color_str

def plot_arc(figure, center, radius, start_angle, end_angle, color, name):
    phi = np.linspace(start_angle, end_angle, 100)
    r = np.ones(phi.shape)*radius
    x = r*np.cos(phi) + center[0]
    y = r*np.sin(phi) + center[1]

    figure.add_trace(go.Scatter(x=x, y=y,
                                mode='lines',
                                name=name + " {:.0f}".format(np.degrees(end_angle-start_angle)),
                                line=dict(color='rgba({}, {})'.format(color, 1.0), width=10., dash='dot'),
                                ))

In [ ]:
def interface_graph(inc_angle, n_inc, n_trans):
    #n_inc = 1.
    #n_trans = 1.5
    inc_angle = np.radians(inc_angle)
    trans_angle = snell(n_inc, inc_angle, n_trans)

    r_s = fresnel_r_s(n_inc, inc_angle, n_trans, trans_angle)
    r_p = fresnel_r_p(n_inc, inc_angle, n_trans, trans_angle)

    t_s = fresnel_t_s(n_inc, inc_angle, n_trans, trans_angle)
    t_p = fresnel_t_p(n_inc, inc_angle, n_trans, trans_angle)

    R = (np.abs(r_s)**2 + np.abs(r_p)**2)*0.5
    R = np.round(R, 3)

    #inc_angle = np.radians(inc_angle)
    #trans_angle = np.radians(trans_angle)

    T = (np.abs(t_s)**2 + np.abs(t_p)**2)*0.5* np.real(n_trans*np.cos(trans_angle)/(n_inc*np.cos(inc_angle)))
    T = np.round(T, 3)
    center_x = -1.5
    center_y = 0
    trans_angle = np.real(trans_angle)
    left_x = -np.cos(inc_angle)*2 + center_x
    left_y_lower = -np.sin(inc_angle)*2 + center_y
    left_y_upper = np.sin(inc_angle)*2 + center_y

    right_x = np.cos(trans_angle)*2 + center_x
    right_y = np.sin(trans_angle)*2 + center_y

    x_inc = [left_x, center_x]
    y_inc = [left_y_lower, center_y]

    x_ref = [center_x, left_x]
    y_ref = [center_y, left_y_upper]

    x_trn = [center_x, right_x]
    y_trn = [center_y, right_y]
    print(x_trn)
    print(y_trn)
    color1 = make_color(127, 201, 127)
    color2 = make_color(190,174,212)
    color3 = make_color(253,192,134)
    alpha1 = 1.
    alpha2 = R
    alpha3 = T

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_inc, y=y_inc,
                        mode='lines',
                         line=dict(color='rgba({}, {})'.format(color1, alpha1),
                                  width=10.),
                        name='Incident'))

    fig.add_trace(go.Scatter(x=x_ref, y=y_ref,
                        mode='lines',
                             line=dict(color='rgba({}, {})'.format(color2, alpha2),
                                       width=10.),
                        name='Reflected: {:.0f}%'.format(alpha2*100)))
    if np.imag(n_trans) == 0:
        fig.add_trace(go.Scatter(x=x_trn, y=y_trn,
                            mode='lines',
                             line=dict(color='rgba({}, {})'.format(color3, alpha3),
                                  width=10.),
                            name='Transmitted: {:.0f}%'.format(alpha3*100)))
    """
    fig.add_shape(type='line',
                    yref="y",
                    xref="x",
                    x0=0.,
                    y0=-2.,
                    x1=0.,
                    y1=2.,
                    line=dict(color='black', width=3))
    """
    
    fig.add_shape(type='line',
                    yref="y",
                    xref="x",
                    x0=-1.5,
                    y0=0.5,
                    x1=1.5,
                    y1=0.5,
                    line=dict(color='black', width=3, dash='dash'))
    
    fig.add_shape(type='line',
                    yref="y",
                    xref="x",
                    x0=-1.5,
                    y0=-0.5,
                    x1=1.5,
                    y1=-0.5,
                    line=dict(color='black', width=3, dash='dash'))

    #plot_arc(fig, (0,0), 0.5, np.pi, np.pi+inc_angle, color1, "Angle Inc.")
    #plot_arc(fig, (0,0), 0.5, np.pi-inc_angle, np.pi, color2, "Angle Ref.")
    #if np.imag(n_trans) == 0.:
    #    plot_arc(fig, (0,0), 0.5, 0., np.real(trans_angle), color3, "Angle Trans.")

    #fig.update_layout(xaxis_showgrid=True, yaxis_showgrid=False, plot_bgcolor='white',
    #                 xaxis_showticklabels=False, yaxis_showticklabels=False)

    fig.update_xaxes(range=[-2., 2.], autorange=False)
    fig.update_yaxes(range=[-1., 1.], autorange=False)
    return fig


In [ ]:
interface_graph(45., 1.5, 1.)

In [ ]:
#fibre = shapely.geometry.box(-1.5, -0.5, 1.5, 0.5)

class Ray():
    
    def __init__(self, intensity, prop_angle, start_point):
        self.intensity = intensity
        self.prop_angle = prop_angle
        self.start_point = start_point
        
    def propagate(self, distance):
        end_point_x = self.start_point[0] + distance*np.cos(self.prop_angle)
        end_point_y = self.start_point[1] + distance*np.sin(self.prop_angle)        
        return np.array([end_point_x, end_point_y])
    
    def back_propagate_forwards(self, distance):
        end_point_x = self.start_point[0] + distance*np.cos(self.prop_angle)
        end_point_y = self.start_point[1] + distance*np.sin(self.prop_angle)        
        return np.array([end_point_x, end_point_y])

start = np.array([0., 0.])
ray = Ray(1.0, np.radians(45.), start)

stop = ray.propagate_forwards(1.)

print(start)
print(stop)
